# intersect the CRISPR guides

**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_REG=${FD_REG}/${FN_REG}
head ${FP_REG}

chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613
chr1	30803	31072
chr1	101603	101849
chr1	115411	115986
chr1	118518	118743
chr1	136071	137429


In [3]:
ASSAY=CRISPRi_FlowFISH
FD_INP=${FD_RES}/results/${ASSAY}
ls ${FD_INP}

coverage  coverage_astarrseq_peak_macs_input


In [4]:
ASSAY=CRISPRi_FlowFISH
FD_INP=${FD_RES}/results/${ASSAY}/coverage
FP_INPS=($(ls ${FD_INP}/*.log2FC_2.filtered.bed))

head ${FD_INP}/Merge_HCRFF_rAVG.log2FC_2.filtered.bed
echo
tail ${FD_INP}/Merge_HCRFF_rAVG.log2FC_2.filtered.bed

chr11	33064196	33064197	id-1495	0.807624	CAPRIN1
chr11	33064217	33064218	id-1496	-2.71246	CAPRIN1
chr11	33064229	33064230	id-1497	0.642045	CAPRIN1
chr11	33064230	33064231	id-1498	1.85354	CAPRIN1
chr11	33064258	33064259	id-1499	0.126467	CAPRIN1
chr11	33064264	33064265	id-1500	0.217096	CAPRIN1
chr11	33064298	33064299	id-1501	-0.612031	CAPRIN1
chr11	33064303	33064304	id-1502	-0.859612	CAPRIN1
chr11	33064315	33064316	id-1503	-0.665188	CAPRIN1
chr11	33064339	33064340	id-1504	-0.569748	CAPRIN1

chr8	128557671	128557672	id-49860	0.622196	PVT1
chr8	128704726	128704727	id-49861	0.351404	PVT1
chr8	128726825	128726826	id-49862	-1.85164	PVT1
chr8	128727502	128727503	id-49863	-2.83422	PVT1
chr8	128727986	128727987	id-49864	-0.467646	PVT1
chr8	128731456	128731457	id-49865	-0.004463	PVT1
chr8	128732458	128732459	id-49866	-0.247074	PVT1
chr8	128732691	128732692	id-49867	-0.33844	PVT1
chr8	128733323	128733324	id-49868	1.28813	PVT1
chr8	128735224	128735225	id-49869	0.53652	PVT1


**RUN**

In [5]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_crispri_hcrff.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=CRISPRi_FlowFISH

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
FD_INP=${FD_RES}/results/${ASSAY}/coverage
FN_INP=Merge_HCRFF_rAVG.log2FC_2.filtered.bed
FP_INP=${FD_INP}/${FN_INP}

FD_OUT=${FD_RES}/results/${ASSAY}/coverage_astarrseq_peak_macs_input
FN_OUT=${FN_INP}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_REG=${FD_REG}/${FN_REG}

### show I/O file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
cat ${FP_INP} | head -5
echo
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
cat ${FP_REG} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -wo \
| gzip -c \
> ${FP_OUT}

### Show I/O file
echo
echo "Output file: " ${FP_OUT}
echo
echo "Show the first few lines of the output file"
zcat ${FP_OUT} | head -n 5
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29603485


**Check results**

In [6]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_crispri_hcrff.0.txt

Hostname:           x2-08-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-24-23+10:18:20

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_FlowFISH/coverage/Merge_HCRFF_rAVG.log2FC_2.filtered.bed

show first few lines of input
chr11	33064196	33064197	id-1495	0.807624	CAPRIN1
chr11	33064217	33064218	id-1496	-2.71246	CAPRIN1
chr11	33064229	33064230	id-1497	0.642045	CAPRIN1
chr11	33064230	33064231	id-1498	1.85354	CAPRIN1
chr11	33064258	33064259	id-1499	0.126467	CAPRIN1

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/CRISPRi_FlowFISH/coverage_astarrseq_peak_macs_input/Merge_HCRFF_rAVG.log2FC_2.filtered.bed.gz

Show the first few l